In [14]:
import ruamel.yaml as yaml
import os
import sys
import pandas as pd
import numpy as np


NO_CONFIG_ERR_MSG = """No config file found. Root directory is determined by presence of "config.yaml" file."""

original_wd = os.getcwd()

# Number of times to move back in directory
num_retries = 10
for x in range(0, num_retries):
    # try to load config file
    try:
        with open("config.yaml", 'r') as stream:
            cfg = yaml.safe_load(stream)
    # If not found move back one directory level
    except FileNotFoundError:
        os.chdir('../')
        # If reached the max number of directory levels change to original wd and print error msg
        if x+1 == num_retries:
            os.chdir(original_wd)
            print(NO_CONFIG_ERR_MSG)

# Add directory to PATH
path = os.getcwd()

if path not in sys.path:
    sys.path.append(path)


## Load Data

In [15]:
RPM_df = pd.read_csv('data/interim/player_RPM_stats.csv')
box_score_df = pd.read_csv('data/raw/Box_Scores.csv')

## Joining RPM to players

Need to join on composite key made up of Player Name + Season

season_id appears to be calendar year the season started in 

Based off of the latest season being 2017 and not 2018

#### Steps:

1) RPM: Seperate names into first and last (start by splitting on a space)

2) Match season_id to either the calendar year the season ended or began. Map to RPM data

3) Join on First, Last, Season

In [16]:
box_score_df['season_id'].unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017], dtype=int64)

In [17]:
RPM_df.head()

,DRPM,GP,MPG,ORPM,RPM,Season_End_Year,Season_Start_Year,WINS,names
0,1.95,62,35.0,6.03,7.98,2014,2013,14.47,Chris Paul
1,0.15,81,38.5,6.27,6.42,2014,2013,17.63,Kevin Durant
2,-0.38,78,36.5,6.62,6.24,2014,2013,15.48,Stephen Curry
3,0.35,68,22.8,5.07,5.42,2014,2013,7.68,Manu Ginobili
4,3.23,69,36.2,1.89,5.12,2014,2013,11.64,LaMarcus Aldridge


In [18]:
# most number of whitespaces 
RPM_df['names'].str.split(' ').apply(len).max()

4

In [19]:
# Who has more than one space?

RPM_df['name_length_post_split'] = RPM_df['names'].str.split(' ').apply(len)
RPM_df.query("name_length_post_split > 2").names.value_counts().head()

# Mostly Jrs and some multiple names

Otto Porter Jr.         5
Tim Hardaway Jr.        5
Luc Mbah a Moute        5
Glenn Robinson III      4
James Michael McAdoo    4
Name: names, dtype: int64

What's the RPM of these players?

In [20]:
RPM_df.query("name_length_post_split > 2").RPM.describe()

count    51.000000
mean     -1.338627
std       2.072766
min      -6.390000
25%      -2.375000
50%      -1.620000
75%      -0.440000
max       4.960000
Name: RPM, dtype: float64

In [21]:
first_name_to_search = 'Tim'
box_score_df[box_score_df.First_Name.str.contains(first_name_to_search)].head()

,Game_id,Person_id,Team_id,First_Name,Last_Name,minutes,Field_Goals,Field_Goals_Attempted,Field_Goal_Percentage,Three_Pointers,...,Fast_Break_Points,Triple_Doubles,Double_Doubles,actual_minutes,actual_seconds,Plus_Minus,Blocks_Against,PTS_OFF_TO,Second_Chance_PTS,Total_Rebounds
28,20300002,1495,1610612759,Timothy,Duncan,40,8,13,0.615,0,...,0.0,0.0,1.0,40,0,-4,1.0,2,6,12.0
173,20300009,1501,1610612749,Timothy,Thomas,30,5,17,0.294,0,...,0.0,0.0,0.0,30,0,-19,2.0,4,4,3.0
210,20300011,1495,1610612759,Timothy,Duncan,34,7,22,0.318,0,...,2.0,0.0,1.0,34,0,-19,2.0,2,6,21.0
408,20300021,1501,1610612749,Timothy,Thomas,32,7,12,0.583,2,...,5.0,0.0,0.0,32,0,17,0.0,2,7,6.0
636,20300032,1501,1610612749,Timothy,Thomas,29,6,15,0.400,2,...,2.0,0.0,0.0,29,0,13,2.0,4,0,4.0


#### For now, will join to get RPM on a season/team/player level and fix this later if necessary

Split Names

In [22]:
RPM_df['First_Name'] = RPM_df['names'].str.split(' ').str.get(0)
RPM_df['Last_Name'] = RPM_df['names'].str.split(' ').str.get(-1)

Rename Starting Calendar year to season_id

In [23]:
RPM_df = RPM_df.rename(columns={'Season_Start_Year':'season_id'})

In [24]:
potential_keys = [
    'First_Name',
    'Last_Name',
    'season_id',
]

## Merge

In [25]:
RPM_player_data = (
    pd.merge(RPM_df, box_score_df, on=potential_keys, how='left')
    .drop_duplicates(subset=potential_keys) # Drop duplicate games, keep only first         
            )


# Check for nulls from box score data
RPM_player_data[RPM_player_data['Total_Rebounds'].isnull()].shape

(74, 50)

In [26]:
RPM_player_data.head()

,DRPM,GP,MPG,ORPM,RPM,Season_End_Year,season_id,WINS,names,name_length_post_split,...,Fast_Break_Points,Triple_Doubles,Double_Doubles,actual_minutes,actual_seconds,Plus_Minus,Blocks_Against,PTS_OFF_TO,Second_Chance_PTS,Total_Rebounds
0,1.95,62,35.0,6.03,7.98,2014,2013,14.47,Chris Paul,2,...,2.0,0.0,1.0,35.0,58.0,-13.0,1.0,2.0,0.0,6.0
75,0.15,81,38.5,6.27,6.42,2014,2013,17.63,Kevin Durant,2,...,10.0,0.0,0.0,39.0,9.0,3.0,1.0,13.0,2.0,6.0
175,-0.38,78,36.5,6.62,6.24,2014,2013,15.48,Stephen Curry,2,...,0.0,0.0,0.0,23.0,57.0,28.0,0.0,0.0,0.0,4.0
260,0.35,68,22.8,5.07,5.42,2014,2013,7.68,Manu Ginobili,2,...,2.0,0.0,0.0,24.0,9.0,10.0,0.0,4.0,0.0,4.0
351,3.23,69,36.2,1.89,5.12,2014,2013,11.64,LaMarcus Aldridge,2,...,2.0,0.0,0.0,39.0,26.0,-16.0,1.0,2.0,0.0,4.0


## Add Minutes Played Season Total to compute a team level average RPM weighted by minutes

In [27]:
RPM_player_data['Season_Total_Minutes'] = RPM_player_data['GP'] * RPM_player_data['MPG'] 
RPM_player_data['Season_Total_Minutes*RPM'] = RPM_player_data['Season_Total_Minutes'] \
* RPM_player_data['RPM']

In [28]:
RPM_player_data.head()

,DRPM,GP,MPG,ORPM,RPM,Season_End_Year,season_id,WINS,names,name_length_post_split,...,Double_Doubles,actual_minutes,actual_seconds,Plus_Minus,Blocks_Against,PTS_OFF_TO,Second_Chance_PTS,Total_Rebounds,Season_Total_Minutes,Season_Total_Minutes*RPM
0,1.95,62,35.0,6.03,7.98,2014,2013,14.47,Chris Paul,2,...,1.0,35.0,58.0,-13.0,1.0,2.0,0.0,6.0,2170.0,17316.600
75,0.15,81,38.5,6.27,6.42,2014,2013,17.63,Kevin Durant,2,...,0.0,39.0,9.0,3.0,1.0,13.0,2.0,6.0,3118.5,20020.770
175,-0.38,78,36.5,6.62,6.24,2014,2013,15.48,Stephen Curry,2,...,0.0,23.0,57.0,28.0,0.0,0.0,0.0,4.0,2847.0,17765.280
260,0.35,68,22.8,5.07,5.42,2014,2013,7.68,Manu Ginobili,2,...,0.0,24.0,9.0,10.0,0.0,4.0,0.0,4.0,1550.4,8403.168
351,3.23,69,36.2,1.89,5.12,2014,2013,11.64,LaMarcus Aldridge,2,...,0.0,39.0,26.0,-16.0,1.0,2.0,0.0,4.0,2497.8,12788.736


## Export all info for players

In [33]:
player_info_export_cols = ['DRPM',
 'GP',
 'MPG',
 'ORPM',
 'RPM',
 'Season_End_Year',
 'season_id',
 'WINS',
 'names',
 'name_length_post_split',
 'First_Name',
 'Last_Name',
 'Game_id',
 'Person_id',
 'Team_id',
 'Season_Total_Minutes',
 'Season_Total_Minutes*RPM']

In [34]:
RPM_player_data[player_info_export_cols].to_csv('data/interim/player-data-RPM-with-identifiers.csv', index=False)

In [42]:
keep_stats = [
 'DRPM',
 'ORPM',
 'RPM',
 'WINS',
]

In [43]:
team_minutes_weighted_avg_RPM = (RPM_player_data.groupby(by=['Team_id', 'season_id'])
 .sum()['Season_Total_Minutes*RPM'] / RPM_player_data.groupby(by=['Team_id', 'season_id'])
 .sum()['Season_Total_Minutes'])
team_minutes_weighted_avg_RPM = (team_minutes_weighted_avg_RPM.to_frame('Team_Minutes_Weighted_Avg_RPM')
                                 .reset_index())

In [44]:
team_RPM_total = (RPM_player_data.groupby(by=['Team_id', 'season_id'])
                  .sum()
                  .loc[:, keep_stats]
                  .add_prefix('Sum_')
                  .reset_index()
)
team_RPM_total.head()

,Team_id,season_id,Sum_DRPM,Sum_ORPM,Sum_RPM,Sum_WINS
0,1.610613e+09,2013,-0.52,-17.03,-17.55,28.48
1,1.610613e+09,2014,5.23,-0.32,4.91,43.37
2,1.610613e+09,2015,8.15,-4.76,3.39,51.35
3,1.610613e+09,2016,6.28,-21.40,-15.12,28.75
4,1.610613e+09,2017,-9.17,-20.81,-29.98,25.26


In [45]:
team_level_RPM_stats = pd.merge(team_minutes_weighted_avg_RPM,
                               team_RPM_total,
                               on=['Team_id', 'season_id'])
team_level_RPM_stats['Team_id'] = team_level_RPM_stats['Team_id'].astype(int)

In [46]:
keep_cols = ['Team_id',
 'Season_id',
             'City',
 'Short_Name',
 'Nickname',]
team_mapping =(pd.read_csv('data/raw/Team_Mapping.csv')
               .loc[:, keep_cols]
               .rename(columns={'Season_id':'season_id'})
               # Remove leading '2' from season id
              .assign(season_id= lambda x: x['season_id'].astype(str).str.slice(1).astype(int)))
team_mapping.head()

,Team_id,season_id,City,Short_Name,Nickname
0,1610612737,2003,Atlanta,Atlanta,Hawks
1,1610612737,2004,Atlanta,Atlanta,Hawks
2,1610612737,2005,Atlanta,Atlanta,Hawks
3,1610612737,2006,Atlanta,Atlanta,Hawks
4,1610612737,2007,Atlanta,Atlanta,Hawks


## Adding team identifiers to data

In [47]:
team_level_RPM_stats = (pd.merge(team_mapping, 
                                team_level_RPM_stats,
                               on=['Team_id', 'season_id'])
                        .drop_duplicates()                                              
                       )
team_level_RPM_stats.head()

,Team_id,season_id,City,Short_Name,Nickname,Team_Minutes_Weighted_Avg_RPM,Sum_DRPM,Sum_ORPM,Sum_RPM,Sum_WINS
0,1610612737,2013,Atlanta,Atlanta,Hawks,0.132475,-0.52,-17.03,-17.55,28.48
2,1610612737,2014,Atlanta,Atlanta,Hawks,1.222454,5.23,-0.32,4.91,43.37
4,1610612737,2015,Atlanta,Atlanta,Hawks,0.962710,8.15,-4.76,3.39,51.35
6,1610612737,2016,Atlanta,Atlanta,Hawks,-0.493510,6.28,-21.40,-15.12,28.75
8,1610612737,2017,Atlanta,Atlanta,Hawks,-1.145967,-9.17,-20.81,-29.98,25.26


## Sorting teams high to low as a sanity check

In [49]:
team_level_RPM_stats.sort_values(by='Team_Minutes_Weighted_Avg_RPM', ascending=False).head(7)

,Team_id,season_id,City,Short_Name,Nickname,Team_Minutes_Weighted_Avg_RPM,Sum_DRPM,Sum_ORPM,Sum_RPM,Sum_WINS
61,1610612744,2016,Golden State,Golden State,Warriors,3.338573,14.21,5.07,19.28,77.38
59,1610612744,2015,Golden State,Golden State,Warriors,2.838597,8.06,8.08,16.14,73.90
57,1610612744,2014,Golden State,Golden State,Warriors,2.760493,11.47,5.09,16.56,64.16
167,1610612759,2015,San Antonio,San Antonio,Spurs,2.586875,19.71,7.79,27.50,66.47
73,1610612745,2017,Houston,Houston,Rockets,2.173224,4.08,-7.10,-3.02,62.18
163,1610612759,2013,San Antonio,San Antonio,Spurs,2.124183,9.76,6.55,16.31,55.07
77,1610612746,2014,Los Angeles,L.A. Clippers,Clippers,2.064357,-5.15,-2.18,-7.33,52.65


In [51]:
team_level_RPM_stats.sort_values(by='Sum_WINS', ascending=False).head(7)

,Team_id,season_id,City,Short_Name,Nickname,Team_Minutes_Weighted_Avg_RPM,Sum_DRPM,Sum_ORPM,Sum_RPM,Sum_WINS
61,1610612744,2016,Golden State,Golden State,Warriors,3.338573,14.21,5.07,19.28,77.38
59,1610612744,2015,Golden State,Golden State,Warriors,2.838597,8.06,8.08,16.14,73.90
167,1610612759,2015,San Antonio,San Antonio,Spurs,2.586875,19.71,7.79,27.50,66.47
57,1610612744,2014,Golden State,Golden State,Warriors,2.760493,11.47,5.09,16.56,64.16
73,1610612745,2017,Houston,Houston,Rockets,2.173224,4.08,-7.10,-3.02,62.18
63,1610612744,2017,Golden State,Golden State,Warriors,1.522624,9.93,0.09,10.02,61.21
176,1610612760,2015,Oklahoma City,Oklahoma City,Thunder,1.583519,-0.29,2.21,1.92,60.25


## Export data

In [53]:
team_level_RPM_stats.to_csv('data/interim/team_level_RPM_stats.csv', index=False)